# 필요 데이터 생성하기

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF

### 1. 위험도 데이터 생성

In [4]:
df = pd.read_csv('Data/교통사망사고정보/original_kor.csv',encoding='CP949',engine='python')

# temp) 총부상자수 변수
df['총부상자수'] = df['사상자수'] - df['사망자수']

# 발생지시군구_위험도
total= df.groupby(['발생지시도','발생지시군구']).count().주야.sort_index()
a = df.groupby(['발생지시도','발생지시군구']).sum().사망자수.sort_index()
b = df.groupby(['발생지시도','발생지시군구']).sum().총부상자수.sort_index()
temp=(a*12+b*3)/total

for i,j in temp.index:
    df.loc[(df.발생지시도==i)&(df.발생지시군구==j),'발생지시군구_위험도'] = temp[i,j]



# 발생지시도x요일x주야_위험도
total = df.groupby(['발생지시도','요일','주야']).count().도로형태.sort_index()
a = df.groupby(['발생지시도','요일','주야']).sum().사망자수.sort_index()
b = df.groupby(['발생지시도','요일','주야']).sum().총부상자수.sort_index()
temp = (a*12+b*3)/total

for i,j,k in temp.index:
    df.loc[(df.발생지시도==i)&(df.요일==j)&(df.주야==k),'발생지시도x요일x주야_위험도'] = temp[i,j,k]

# 차량조합_위험도
total = df.groupby(['당사자종별_1당_대분류','당사자종별_2당_대분류']).count().사고유형_중분류
a = df.groupby(['당사자종별_1당_대분류','당사자종별_2당_대분류']).sum().사망자수.sort_index()
b = df.groupby(['당사자종별_1당_대분류','당사자종별_2당_대분류']).sum().총부상자수.sort_index()
temp = (a*12+b*3)/total

for i,j in temp.index:
    df.loc[(df.당사자종별_1당_대분류==i)&(df.당사자종별_2당_대분류==j),'차량조합_위험도'] = temp[i,j]

# 사고유형_위험도
total = df.groupby(['사고유형_중분류']).count().당사자종별_1당_대분류
a = df.groupby(['사고유형_중분류']).sum().사망자수.sort_index()
b = df.groupby(['사고유형_중분류']).sum().총부상자수.sort_index()
temp = (a*12+b*3)/total

for i in temp.index:
    df.loc[(df.사고유형_중분류==i),'사고유형_위험도'] = temp[i]


temp = df[['발생지시도','발생지시군구','요일','주야','당사자종별_1당_대분류','당사자종별_2당_대분류',
   '사고유형_중분류','발생지시군구_위험도','발생지시도x요일x주야_위험도','차량조합_위험도',
   '사고유형_위험도']]

temp.to_csv('Data/plus_danger.csv',encoding='CP949',index=False)

### 2. NMF 데이터 생성

In [6]:
df = pd.read_csv('Data/교통사망사고정보/original_kor.csv',encoding='CP949',engine='python')

df['발생지'] = df['발생지시도'] + df['발생지시군구']
df['발생지'].replace('경남창원시(통합)','경남창원시', inplace=True)

df['요일/사고유형_중'] = df['요일'] + '_'+df['사고유형_중분류']

day_acc = df.groupby(['발생지','요일/사고유형_중'])['주야'].count().unstack().fillna(0)

X = day_acc.values

model = NMF(n_components=3, init='random', random_state=0)
W = model.fit_transform(X)
df_nmf = pd.DataFrame(W)
df_nmf.rename(columns={0:'day_acc_nmf1',1:'day_acc_nmf2',2:'day_acc_nmf3'}, inplace=True)
df_nmf = df_nmf.join(pd.DataFrame(day_acc.index))

df = df.merge(df_nmf, on='발생지')
df.drop('요일/사고유형_중',axis=1,inplace=True)

temp = df[['day_acc_nmf1','day_acc_nmf2','day_acc_nmf3','발생지']]

temp.columns = ['요일_사고유형_NMF1','요일_사고유형_NMF2','요일_사고유형_NMF3','발생지']
temp = temp.groupby('발생지').mean()
temp.to_csv('Data/plus_nmf.csv',encoding='CP949',index=True)

### 3. 사고다발지 데이터 생성

In [7]:
## 사고다발지 데이터 생성

df_jaywalk = pd.read_csv('Data/보조데이터/무단횡단사고다발지(2012~2016).csv',encoding='CP949',engine='python')
df_elder = pd.read_csv('Data/보조데이터/보행노인사고다발지(2012~2016).csv',encoding='CP949',engine='python')
df_child = pd.read_csv('Data/보조데이터/보행어린이사고다발지(2012~2016).csv',encoding='CP949',engine='python')
df_school = pd.read_csv('Data/보조데이터/스쿨존내어린이사고다발지(2012~2016).csv',encoding='CP949',engine='python')
df_bike = pd.read_csv('Data/보조데이터/자전거사고다발지(2012~2016).csv',encoding='CP949',engine='python')

df_jaywalk["사고피해정도"] = df_jaywalk.사망자수*4 + df_jaywalk.중상자수*3 + df_jaywalk.경상자수*2 + df_jaywalk.부상신고자수*1
df_elder["사고피해정도"] = df_elder.사망자수*4 + df_elder.중상자수*3 + df_elder.경상자수*2 + df_elder.부상신고자수*1
df_child["사고피해정도"] = df_child.사망자수*4 + df_child.중상자수*3 + df_child.경상자수*2 + df_child.부상신고자수*1
df_bike["사고피해정도"] = df_bike.사망자수*4 + df_bike.중상자수*3 + df_bike.경상자수*2 + df_bike.부상신고자수*1
df_school["사고피해정도"] = df_school.사망자수*4 + df_school.중상자수*3 + df_school.경상자수*2 + df_school.부상신고자수*1

def replace_region(df):    
    df = df.groupby('다발지명').agg({'사고피해정도': 'sum', '발생건수':'sum'})
    df.reset_index(inplace=True)
    df['시도'] = df['다발지명'].apply(lambda x: x.split(' ')[0])
    df['시군구'] = df['다발지명'].apply(lambda x: x.split(' ')[1])
    df['시도'].replace(['경상남도', '경상북도', '전라남도', '전라북도', '충청남도', '충청북도'],
                                   ['경남', '경북', '전남', '전북', '충남', '충북'], inplace=True)
    df['시도'] = df['시도'].apply(lambda x: x[:2])
    df.drop('다발지명', axis=1, inplace=True)
    df['시군구'].replace('세종특별자치시','세종', inplace=True)
    df['발생지'] = df['시도'] + df['시군구']
    df = df.groupby(['발생지'])['사고피해정도', '발생건수'].sum()
    df.reset_index(inplace=True)
    return df
    
walk_list = ['Data/plus_mudan','Data/plus_old','Data/plus_child','Data/plus_school','Data/plus_bic']
i = 0
for x in [df_jaywalk, df_elder, df_child, df_school, df_bike]:
    df = replace_region(x)
    df.to_csv(walk_list[i]+'.csv',index=False)
    i += 1

# 피쳐추가하기

In [10]:
## 기존 데이터 불러오기 ###
df = pd.read_csv('Data/교통사망사고정보/original_kor.csv',encoding='CP949',engine='python')
df = df[['주야', '요일', '사망자수', '사상자수', '중상자수', '경상자수', '부상신고자수', '발생지시도', '발생지시군구',
       '사고유형_대분류', '사고유형_중분류', '법규위반', '도로형태_대분류', '도로형태', '당사자종별_1당_대분류',
       '당사자종별_2당_대분류']]


### 추가 데이터 불러오기 ###
# 고속도로 데이터 #
highway = pd.read_csv('Data/plus_highway.csv',encoding='CP949')
highway['총길이']=np.float64(highway.총길이)

# 사고다발지 데이터 #
jaywalk = pd.read_csv('Data/plus_mudan.csv')
elder = pd.read_csv('Data/plus_old.csv')
child = pd.read_csv('Data/plus_child.csv')
bike = pd.read_csv('Data/plus_bic.csv')
school = pd.read_csv('Data/plus_school.csv')

jaywalk.columns = ['발생지', '무단횡단피해', '무단횡단건수']
elder.columns = ['발생지', '노인피해', '노인건수']
child.columns = ['발생지', '어린이피해', '어린이건수']
bike.columns = ['발생지', '자전거피해', '자전거건수']
school.columns = ['발생지', '학교피해', '학교건수']

# 위험도 데이터 #
danger = pd.read_csv('Data/plus_danger.csv',encoding='CP949')

# nmf 데이터 #
nmf = pd.read_csv('Data/plus_nmf.csv',encoding='CP949')

In [11]:
### 기존데이터 전처리 ###

df.loc[df.발생지시군구=='연기군','발생지시군구']='세종'
df.loc[(df.발생지시도=='충남')&(df.발생지시군구=='세종'),'발생지시도']='세종'

# 충북 청원군 -> 충북 청주시 변경 (14년 7월)
df.loc[df.발생지시군구=='청원군','발생지시군구']='청주시'
    
#발생지 변수
df['발생지']=df['발생지시도']+df['발생지시군구']
df['발생지'].replace('경남창원시(통합)','경남창원시',inplace=True)

In [12]:
def change(df, highway, jaywalk, elder, child, bike, school, danger, nmf ):
    ### 고속도로데이터 ###
    df=pd.merge(df,highway,on='발생지',how='left')

    ### 사고다발지데이터 ###
    for x in  [jaywalk, elder, child, bike, school]:
        df = df.merge(x, on='발생지', how='left')
    df.drop(df[(df.주야.isnull())&(df.요일.isnull())].index,inplace=True)

    df.loc[(df.발생지.isnull()==False) & (df.무단횡단피해.isnull()==True), ['무단횡단피해', '무단횡단건수']]=0
    df.loc[(df.발생지.isnull()==False) & (df.노인피해.isnull()==True), ['노인피해', '노인건수']]=0
    df.loc[(df.발생지.isnull()==False) & (df.어린이피해.isnull()==True), ['어린이피해','어린이건수']]=0
    df.loc[(df.발생지.isnull()==False) & (df.자전거피해.isnull()==True), ['자전거피해','자전거건수']]=0
    df.loc[(df.발생지.isnull()==False) & (df.학교피해.isnull()==True), ['학교피해','학교건수']]=0
    
    ### nmf 데이터 ###
    df = df.merge(nmf, on='발생지',how='left')

    ### 위험도 데이터 ###
    # 발생지시군구 위험도
    temp = danger.groupby(['발생지시도','발생지시군구']).mean().발생지시군구_위험도
    for i,j in temp.index:
        df.loc[(df.발생지시도==i)&(df.발생지시군구==j),'발생지시군구_위험도'] = temp[i,j]

    # 발생지시도x요일x주야 위험도
    temp = danger.groupby(['발생지시도','요일','주야']).mean().발생지시도x요일x주야_위험도
    for i,j,k in temp.index:
        df.loc[(df.발생지시도==i)&(df.요일==j)&(df.주야==k),'발생지시도x요일x주야_위험도'] = temp[i,j,k]

    # 차량조합 위험도
    temp = danger.groupby(['당사자종별_1당_대분류','당사자종별_2당_대분류']).mean().차량조합_위험도
    for i,j in temp.index:
        df.loc[(df.당사자종별_1당_대분류==i)&(df.당사자종별_2당_대분류==j),'차량조합_위험도'] = temp[i,j]

    # 사고유형 위험도
    temp = danger.groupby(['사고유형_중분류']).mean().사고유형_위험도
    for i in temp.index:
        df.loc[(df.사고유형_중분류==i),'사고유형_위험도'] = temp[i]

    # 사고위험 평균
    df['위험도평균'] = np.nanmean(np.array(df[['발생지시군구_위험도','발생지시도x요일x주야_위험도','차량조합_위험도','사고유형_위험도']]),axis=1)

    
    return df

In [13]:
df = change(df, highway, jaywalk, elder, child, bike, school, danger, nmf )